Implementation of the base model (June 12)
---

In [155]:
from gurobipy import Model, GRB, quicksum
import numpy as np
import pandas as pd
import openpyxl

## 0 Instances

### Read the file from local directory

Arthur's:

'/Users/arthurdebelle/Desktop/TUM/SoSe 2024/Ad.S - OM/Project/CODING/Airports data/Brussels (EBBR)/Brussels_Clean.xlsm'

Ting-Ying's:

'/Users/chentingying/Documents/tum/Ad_Se_Operation_Management/Airports_data/Brussels_Clean.xlsm'

In [ ]:
''' For now: change local_path to represent acctual directory'''

local_path = '/Users/chentingying/Documents/tum/Ad_Se_Operation_Management/Airports_data/Brussels_Clean.xlsm'

# Reading the number of flights Brussels airport
flightCount = len(pd.read_excel(local_path, sheet_name='EBBR - Flights', usecols='A:T', skiprows=2))
flightsBrussels = pd.read_excel(local_path, usecols='A:T', skiprows=1, nrows=flightCount)
print("flightCount: ", flightCount)

# Reading the number of Gates from Brussels airport
gateCount = len(pd.read_excel(local_path, sheet_name='EBBR - Gates', usecols='A:D', skiprows=1))
gatesBrussels = pd.read_excel(local_path, sheet_name='EBBR - Gates', usecols='A:D', skiprows=0, nrows=gateCount)
print("gateCount: ", gateCount)
'''add a dummy gate?'''

### Parameters

In [ ]:
'''
# n = number of flights, m = number of gates
# T = matrix of time differences, P = preferences matrix
# alpha1, alpha2, alpha3 = weight factors, t_max = maximum buffer time
# U = successor function, M = valid gate assignments for each flight
# '''

# Inputs
num_flights = flightCount # Number of flights
num_gates = gateCount + 1 # Number of real gates + 1 dummy gate (Should I +1 or not?)

# Flight columns
Flight_No = flightsBrussels['Flight']   # Aircrafts' flight number @ arrival
ETA = flightsBrussels['ETA']
ETD = flightsBrussels['ETD']
E_Parking = flightsBrussels['Planned Duration']
RTA = flightsBrussels['RTA']
RTD = flightsBrussels['RTD']
R_Parking = flightsBrussels['Real Duration']
Tot_Delay = flightsBrussels['Total Delay']
AC_size = flightsBrussels['AC size (m)']

# Tests
FN2 = Flight_No.tolist()

# Gates columns
Gate_Name = gatesBrussels['Name']
Max_Wingspan = gatesBrussels['Max length (m)']   # Max wingspan allowed on that gate

'''No idea what this is'''
# matrix of time differences of arrivals
# buffer time
# T = [
#     [0, 10, 15, 20],  # Flight 1 can sequentially follow Flights 2, 3, 4
#     [10, 0, 15, 25],  # Flight 2 can sequentially follow Flights 1, 3, 4
#     [15, 15, 0, 30],  # Flight 3 can sequentially follow Flights 1, 2, 4
#     [20, 25, 30, 0]   # Flight 4 can sequentially follow Flights 1, 2, 3
# ]

# Building U -> U[i] = successor of i (=0 if no successor)
U = []
for i in range(num_flights):
    U.extend([3*i+1, 3*i+2, 0])     # U = [1,2,0, 4,5,0, ...]
# print(f"U = {U}")

'''
i = 0,1,2 = "Landing", "Parking" and "Departure" index for the 1st flight of the day; i = 3,4,5 = ...
U[1] = 2    -> Parking of 1st flight has Departure of 1st flight as successor
U[5] = 0    -> Departure of 2nd flight has no successor
i%3 = 0     -> landing of a flight x,     with x = (i-i%3)/3 
i%3 = 1     -> parking of a flight x,     with x = (i-i%3)/3 
i%3 = 2     -> departure of a flight x,   with x = (i-i%3)/3 
'''

# Building M
M = Gate_Name.tolist()
'''
For now: M = [0,1,2,...] = allowed gates for ALL flights
Later: make it [[0, 3, 50, ...], ...] when we have preferences for each flights individually
'''

alpha1 = 1   # Preference scaling factor
alpha2 = 20  # Reward for avoiding tows
alpha3 = 100  # Penalty scaling factor for buffer time deficits

t_max = 30

'''
shadow_constraints should be a list of 4-tuples (i, k, j, l) 
indicating that activity i can’t occur at gate k while activity j occurs at gate l.
'''
shadow_constraints = [
    # Constraint to avoid timing overlaps
    (0, 0, 1, 0),  # If Flight 1 is at Gate 1, Flight 2 should not be at Gate 1 due to overlap
    (1, 1, 2, 1),  # If Flight 2 is at Gate 2, Flight 3 should not be at Gate 2
    (3, 0, 4, 0),  # If Flight 4 is at Gate 1, Flight 5 should not be at Gate 1
    (5, 2, 0, 2),  # If Flight 6 is at Gate 3, Flight 1 should not be at Gate 3

    # Constraints to restrict the usage of Dummy Gate (assuming index 2 as Dummy Gate)
    (0, 2, 1, 2),  # If Flight 1 uses Dummy Gate, no other flights should use it
    (1, 2, 2, 2),  # If Flight 2 uses Dummy Gate, no other flights should use it
    (3, 2, 4, 2),  # If Flight 4 uses Dummy Gate, no other flights should use it

    # Specific restrictions for other logistical reasons
    (2, 1, 3, 1),  # If Flight 3 is at Gate 2, Flight 4 should not be at Gate 2
    (4, 0, 5, 0),  # If Flight 5 is at Gate 1, Flight 6 should not be at Gate 1
]

### Example Inputs

run this to test the following codes

In [156]:
# Example Inputs with Updated Constraints for Dummy Gate

# Define flights and gates
flights = ['Flight 1', 'Flight 2', 'Flight 3', 'Flight 4', 'Flight 5', 'Flight 6']
num_flights = len(flights)  # Number of flights

gates = ['Gate 1', 'Gate 2', 'Gate 3', 'Dummy Gate']
num_gates = len(gates)  # Number of real gates plus one dummy gate for overflow

# Define time differences indicating when one flight can sequentially follow another
T = [
    [0, 15, -10, 25, 30, 20],   # Flight 1
    [15, 0, 20, -5, 25, 30],    # Flight 2
    [-10, 20, 0, 30, -5, 25],   # Flight 3
    [25, -5, 30, 0, 20, -10],   # Flight 4
    [30, 25, -5, 20, 0, 30],    # Flight 5
    [20, 30, 25, -10, 30, 0]    # Flight 6
]

# Updated preferences for each flight regarding each gate, improving the attractiveness of the Dummy Gate
P = [
    [60, 50, 40, 45],  # Flight 1 now has a better preference for the Dummy Gate
    [70, 60, 50, 55],  # Flight 2
    [30, 80, 60, 65],  # Flight 3
    [40, 30, 70, 35],  # Flight 4
    [50, 40, 20, 45],  # Flight 5
    [60, 45, 35, 50]   # Flight 6
]

U = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]  # Successor function for each flight and gate

# Specify valid gate assignments for each flight (inclusive of the Dummy Gate without capacity constraints)
M = [
    [0, 1, 2, 3],    # Flight 1 can go to any gate including Dummy
    [0, 1, 2, 3],    # Flight 2
    [0, 1, 2, 3],    # Flight 3
    [0, 1, 2, 3],    # Flight 4
    [0, 1, 2, 3],    # Flight 5
    [0, 1, 2, 3]     # Flight 6
]

alpha1 = 10   # Preference scaling factor
alpha2 = 50   # Reward for avoiding tows
alpha3 = 500  # High penalty scaling factor for buffer time deficits

t_max = 30

# Shadow constraints to ensure operational compliance
shadow_constraints = [
    (0, 0, 1, 0),  # Conflict between Flight 1 at Gate 1 and Flight 2 at Gate 1
    (1, 1, 4, 1),  # Conflict between Flight 2 at Gate 2 and Flight 5 at Gate 2
    (2, 2, 5, 2),  # Conflict between Flight 3 at Gate 3 and Flight 6 at Gate 3
    # No shadow constraints for the Dummy Gate as it has no capacity constraints
]


### Vertices and Weights

In [157]:
# Calculate and store vertices and weight into variables

def get_vertices(num_flights, num_gates):
    return num_flights + num_gates -1 # Excludes dummy gate

vertices = get_vertices(num_flights, num_gates)
print("vertices: ", vertices)

def get_weight_matrix(num_flights, m, T, P, U, M, alpha1, alpha2, alpha3, t_max):
    
    # Initialize the edge weights matrix
    weights = [[0] * vertices for _ in range(vertices)]
    '''weights = [0]*len(a)''' #But "a" in an int, and object of type 'int' has no len()
    large_negative = -1e6  # A large negative number to represent -inf

    # Populate the weights matrix based on given rules (6)
    for i in range(num_flights):
        for j in range(num_flights):
            if i != j:
                if T[i][j] < 0: # Activities overlap in time
                    weights[i][j] = large_negative
                    '''What will be a recommanded way to handle -inf? 
                    -inf is not allowed in weight matrix
                    Because I can't multiply and sum them in the objective function'''

                elif U[i] == j or U[j] == i: # Saving a tow
                    weights[i][j] = alpha2  
                else:
                    weights[i][j] = -alpha3 * max(t_max - T[i][j], 0)  # Buffer time difference
    
    # Weights for flight to gate assignments (7)
    for i in range(num_flights):
        for j in range(num_flights, vertices):
            if j - num_flights in M[i]:
                weights[i][j] = alpha1 * P[i][j - num_flights]
            else:
                weights[i][j] = large_negative
    
    # Gates cannot be in the same clique (8)
    for i in range(num_flights, vertices):
        for j in range(num_flights, vertices):
            weights[i][j] = large_negative

    return weights


weights = get_weight_matrix(num_flights, num_gates, T, P, U, M, alpha1, alpha2, alpha3, t_max)
print("length of weights: ", len(weights))

vertices:  9
length of weights:  9


2 MIP of Clique Partitioning Problem (CPP)
---

Assumptions
1. Flights are represented as nodes in a graph.
2. Conflicts between flights are represented as edges.
3. Each clique represents a set of flights that can be assigned to the same gate without conflict.

### Callback Function

In [158]:
# Callback function to add lazy constraints
'''
Denote by (x_ij)^* the set of all binary variables x_ij that are set to 1 in your current optimal solution. 
You can then cut off this solution by adding the constraint sum_((i,j): x_ij^* = 1) x_ij <= |(x_ij)^*| - 1, 
where the latter expression is the number of nonzero variables in your current optimal solution.
'''

def mycallback(model, where):
    if where == GRB.Callback.MIPSOL:
        violated_constraints = 0
        for (i, k, j, l) in model._shadow_constraints:
            if (i, k) in model._x and (j, l) in model._x:
                if model.cbGetSolution(model._x[i, k]) > 0.5 and model.cbGetSolution(model._x[j, l]) > 0.5:
                    model.cbLazy(model._x[i, k] + model._x[j, l] <= 1)
                    violated_constraints += 1

### CPP Builder

In [159]:
def build_cpp_model(vertices, weights):

    # Define the Gurobi model and set parameters
    model = Model("GateAssignmentCPP")
    model.setParam(GRB.Param.LazyConstraints, 1)  # Enable lazy constraints

    # Decision variables: x[i, j] = 1 if i and j are in the same clique, 0 otherwise
    x = {}
    for i in range(vertices):
        for j in range(i + 1, vertices):
            x[i, j] = model.addVar(vtype=GRB.BINARY, name=f"x[{i},{j}]")
    
    # Objective: Maximize the sum of the weights for edges within the same clique
    model.setObjective(quicksum(weights[i][j] * x[i, j] for i in range(vertices) for j in range(i + 1, vertices)), GRB.MAXIMIZE)

    # Add transitivity constraints for forming valid cliques (4)
    for i in range(vertices):
        for j in range(vertices):
            for k in range(vertices):
                #''' 1 <= i and k <= a are not needed. Your iterator for i starts at 0 and k <= a holds by definition.'''
                if i < j and j < k:
                    model.addConstr(x[i, j] + x[j, k] - x[i, k] <= 1)
                    model.addConstr(x[i, j] - x[j, k] + x[i, k] <= 1)
                    model.addConstr(-x[i, j] + x[j, k] + x[i, k] <= 1)

    # Store variables and constraints for the callback
    model._x = x
    model._shadow_constraints = shadow_constraints
    # Optimize with the callback function to add lazy constraints
    model.optimize(mycallback)

    # Prepare cluster assignments from the solution
    cliques = {}
    if model.status == GRB.Status.OPTIMAL:
        solution = model.getAttr('x', x)
        for (i, j), value in solution.items():
            if value > 0.5:
                if i not in cliques:
                    cliques[i] = []
                if j not in cliques:
                    cliques[j] = []
                cliques[i].append(j)
                cliques[j].append(i)

        # Generate clusters from cliques
        C = [None] * vertices  # Initialize cluster assignments
        cluster_id = 0
        for key, value in cliques.items():
            if C[key] is None:  # Assign new cluster id if not already assigned
                for v in value:
                    if C[v] is None:
                        C[v] = cluster_id
                C[key] = cluster_id
                cluster_id += 1

    return model, x, C

### Solution Extraction 

In [160]:
# Build the model and retrieve the solution and cluster assignments
model, x, C = build_cpp_model(vertices, weights)

# Check if the model found an optimal solution
if model.status == GRB.Status.OPTIMAL:
    print("\nOptimal solution found with total score:", model.objVal)
    
    # Retrieve the solution from the model
    solution = model.getAttr('x', x)
    cliques = {}  # To store cliques formed in the solution

    # Process each pair in the solution
    for (i, j), value in solution.items():
        if value > 0.5:  # If this pair is in the same clique
            if i in cliques:
                cliques[i].append(j)
            else:
                cliques[i] = [j]
            if j in cliques:
                cliques[j].append(i)
            else:
                cliques[j] = [i]

    # Print details about each clique, separating flights and gates
    for i in range(num_flights):  # Iterate over flight indices
        if i in cliques:
            clique = [i] + cliques[i]
            flights = [f"Flight {x+1}" for x in clique if x < num_flights]
            gates = [f"Gate {x-num_flights+1}" for x in clique if x >= num_flights]
            print(f"Flight {i+1} is in a clique with {', '.join(flights + gates)}")
        else:
            print(f"Flight {i+1} did not obtain any particular gate assignment")

    for i in range(num_flights, num_flights + num_gates):  # Iterate over gate indices
        if i in cliques:
            clique = [i] + cliques[i]
            gates = [f"Gate {x-num_flights+1}" for x in clique if x >= num_flights]
            flights = [f"Flight {x+1}" for x in clique if x < num_flights]
            print(f"Gate {i-num_flights+1} is in a clique with {', '.join(gates + flights)}")

    # Print the final cluster assignments
    print("\nCluster Assignments (C):", C)

else:
    print("No optimal solution found.")


Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 252 rows, 36 columns and 756 nonzeros
Model fingerprint: 0x6c8ff181
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1800.0000000
Presolve time: 0.00s
Presolved: 252 rows, 36 columns, 756 nonzeros
Variable types: 0 continuous, 36 integer (36 binary)
Found heuristic solution: objective 3450.0000000
Root relaxation presolved: 147 rows, 28 columns, 396 nonzeros


Root relaxation: objective 4.450000e+03, 13 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

## 3 Ejection Chain Algorithm

### Heuristic Calculation for Vertex Movement

In [161]:
def calculate_heuristic(vertices, weights):
    """
    Calculate the heuristic value for moving vertex i from its current cluster C(i) to a new cluster D.
    
    Parameters:
        i (int): The vertex to be moved.
        C (list): The current clusters of all vertices.
        D (int): The new cluster to which vertex i will be moved.
        weights (list of lists): The adjacency matrix representing the weights between vertices.
        vertices (int): Total number of vertices.
        
    Returns:
        float: The heuristic value of the move.
    """
    # Current cluster of vertex i
    C_i = C[i]
    
    # Sum of weights of edges between vertex i and all vertices in the new cluster D
    sum_weights_new_cluster = sum(weights[i][j] for j in range(vertices) if C[j] == D)
    
    # Sum of weights of edges between vertex i and all vertices in its current cluster C(i), excluding i itself
    sum_weights_current_cluster = sum(weights[i][j] for j in range(vertices) if C[j] == C_i and j != i)
    
    # Heuristic value h(i, C(i), D)
    h_value = sum_weights_new_cluster - sum_weights_current_cluster
    
    return h_value


# Example usage of heuristic function
vertices = get_vertices(num_flights, num_gates)
weights = get_weight_matrix(num_flights, num_gates, T, P, U, M, alpha1, alpha2, alpha3, t_max)

# Current cluster assignments for all vertices
_, _, C = build_cpp_model(vertices, weights)
print("Current cluster C: ", C)

# Loop to calculate heuristic values for every vertex i and every possible neighbor D
for i in range(num_flights):
    for D in range(num_gates-1):
        if D != C[i]:  # Ensure that we do not consider moving a vertex to its current cluster
            h_value = calculate_heuristic(vertices, weights)
            print(f"Heuristic value for moving vertex {i} (Flight {i+1}) to vertex {num_flights+D} (Gate {D+1}): {h_value}")

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 252 rows, 36 columns and 756 nonzeros
Model fingerprint: 0x6c8ff181
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1800.0000000
Presolve time: 0.00s
Presolved: 252 rows, 36 columns, 756 nonzeros
Variable types: 0 continuous, 36 integer (36 binary)
Found heuristic solution: objective 3450.0000000
Root relaxation presolved: 147 rows, 28 columns, 396 nonzeros


Root relaxation: objective 4.450000e+03, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

### Algorithm 2: Initial Solution Generation

In [162]:
def algorithm_2(alg2_initial):
    """
    Generate an initial solution, trying to avoid assigning flights to the dummy gate unless necessary.
    
    Parameters:
        vertices (int): Total number of vertices (flights + gates).
        weights (list of lists): Weight matrix representing interaction between vertices.
        U (list): List containing successor information for each flight.
        initial_solution (list): Starting solution template, typically with no assignments.

    Indices note:
        Vertex i represents Floght i+1
        Cluster i represents Gate i+1
        
    Returns:
        list: Updated initial solution with minimized use of the dummy gate.
    """
    
    # Initial solution setup: No two vertices are related
    solution = alg2_initial[:]
    print("alg2_initial:", alg2_initial)
    
    # Set all vertices as non-tabu initially
    non_tabu = set(range(num_flights))
    
    # Iterate until there are non-tabu vertices to process
    while non_tabu:
        best_improvement = float('-inf')
        best_i, best_D = None, None
        
        # Iterate over all non-tabu vertices
        for i in non_tabu:
            if U[i] != 0 and U[i] not in non_tabu:
                continue  # Skip if the vertex is a successor and the successor is tabu
            current_cluster = solution[i]
            print()
            print(f"Checking moves form Vertex {i} from Cluster {current_cluster}")

            # Evaluate all possible clusters D for this vertex
            for D in range(num_gates):
                if D == current_cluster:
                    continue
                
                # Calculate the potential heuristic improvement
                improvement = calculate_heuristic(vertices, weights)
                print(f"Evaluating move of Vertex {i} to Cluster {D}: Improvement {improvement}")

                if improvement > best_improvement:
                    best_improvement = improvement
                    best_i, best_D = i, D
                    print(f"New best move found: Move Vertex {i} to Cluster {D} with improvement {improvement}")

        
        if best_i is not None:
            # Assign the best vertex to the best cluster and mark it as tabu
            solution[best_i] = best_D
            non_tabu.remove(best_i)
            print(f"Moving Vertex {best_i} to Cluster {best_D} , marking Vertex {best_i} as tabu.")
        
        else:
            print("No improving move found, exiting loop.")
            break  # Exit if no improving move is found
    
    # Assign all flights without a gate to the dummy gate
    dummy_gate_index = vertices - 1  # Assuming the last index is the dummy gate
    for i in range(vertices):
        if solution[i] is None:
            solution[i] = dummy_gate_index
    
    return solution

# Example usage:
alg2_initial = C  
# alg2_initial = [None] * vertices

# Run algorithm 2
alg2_solution = algorithm_2(alg2_initial)
print()
print("Solution after applying Algorithm 2 (alg2_solution):", alg2_solution)

alg2_initial: [0, 1, 2, 2, 0, 1, 1, 0, 2]

Checking moves form Vertex 0 from Cluster 0
Evaluating move of Vertex 0 to Cluster 1: Improvement -1002750.0
New best move found: Move Vertex 0 to Cluster 1 with improvement -1002750.0
Evaluating move of Vertex 0 to Cluster 2: Improvement -1002750.0
Evaluating move of Vertex 0 to Cluster 3: Improvement -1002750.0

Checking moves form Vertex 1 from Cluster 1
Evaluating move of Vertex 1 to Cluster 0: Improvement -1002750.0
Evaluating move of Vertex 1 to Cluster 2: Improvement -1002750.0
Evaluating move of Vertex 1 to Cluster 3: Improvement -1002750.0

Checking moves form Vertex 2 from Cluster 2
Evaluating move of Vertex 2 to Cluster 0: Improvement -1002750.0
Evaluating move of Vertex 2 to Cluster 1: Improvement -1002750.0
Evaluating move of Vertex 2 to Cluster 3: Improvement -1002750.0

Checking moves form Vertex 3 from Cluster 2
Evaluating move of Vertex 3 to Cluster 0: Improvement -1002750.0
Evaluating move of Vertex 3 to Cluster 1: Improvemen

### Algorithm 1: Core Ejection Chain Algorithm

In [163]:
def algorithm_1(alg1_initial):
    import random

    # Ensure every vertex has a default initial cluster if not assigned
    solution = alg1_initial[:]
    print("alg1_initial:", alg1_initial)
    tabu_list = set()
    improved = True
    iteration = 0
    last_best_score = float('-inf')

    while improved: 
        improved = False
        best_move = None
        best_improvement = float('-inf')

        for i in range(num_flights):
            if i in tabu_list:
                continue

            current_cluster = solution[i]
            # Iterate through all potential new clusters for vertex i
            for D in range(num_gates):
                if D != current_cluster and D not in tabu_list:
                    current_improvement = calculate_heuristic(vertices, weights)
                    # Logging each evaluation
                    print(f"Evaluating move of Vertex {i} from Cluster {current_cluster} to Cluster {D}: Improvement {current_improvement}")

                    if current_improvement > best_improvement:
                        best_improvement = current_improvement
                        best_move = (i, D)

        if best_move:
            i, D = best_move
            solution[i] = D
            tabu_list.add(i)
            improved = True
            print(f"Moving Vertex {i} to Cluster {D}, marking Vertex {i} as tabu.")

            if len(tabu_list) > vertices / 2:
                tabu_list.clear()
                print("Tabu list cleared due to size.")

        iteration += 1
        print(f"Iteration {iteration} completed with solution: {solution}")
        print()

    return solution

# Usage example
alg1_initial = alg2_solution
alg1_solution = algorithm_1(alg2_solution)
print("Final solution:", alg1_solution)


alg1_initial: [1, 0, 0, 0, 1, 0, 1, 0, 2]
Evaluating move of Vertex 0 from Cluster 1 to Cluster 0: Improvement -1002750.0
Evaluating move of Vertex 0 from Cluster 1 to Cluster 2: Improvement -1002750.0
Evaluating move of Vertex 0 from Cluster 1 to Cluster 3: Improvement -1002750.0
Evaluating move of Vertex 1 from Cluster 0 to Cluster 1: Improvement -1002750.0
Evaluating move of Vertex 1 from Cluster 0 to Cluster 2: Improvement -1002750.0
Evaluating move of Vertex 1 from Cluster 0 to Cluster 3: Improvement -1002750.0
Evaluating move of Vertex 2 from Cluster 0 to Cluster 1: Improvement -1002750.0
Evaluating move of Vertex 2 from Cluster 0 to Cluster 2: Improvement -1002750.0
Evaluating move of Vertex 2 from Cluster 0 to Cluster 3: Improvement -1002750.0
Evaluating move of Vertex 3 from Cluster 0 to Cluster 1: Improvement -1002750.0
Evaluating move of Vertex 3 from Cluster 0 to Cluster 2: Improvement -1002750.0
Evaluating move of Vertex 3 from Cluster 0 to Cluster 3: Improvement -1002750.

### Algorithm 3: Putting It All Together

Others
---

### 1 Flight-Gate Scheduling (FGS) Problem

not the point, can come back later

In [ ]:
# Wait for update 

# Callback function to add lazy constraints
def mycallback(model, where):
    if where == GRB.Callback.MIPSOL:
        for (i, j) in model._shadow_constraints:
            if (i, j) in model._x and model.cbGetSolution(model._x[i, j]) > 0.5:
                model.cbLazy(model._x[i, j] == 0)

def build_FGS_model(n, m, P, U, T, M, shadow_restrictions, alpha1, alpha2, alpha3, t_max):
    model = Model("FlightGateScheduling")
    model.setParam(GRB.Param.LazyConstraints, 1)  # Enable lazy constraints

    # Decision variables for each activity being assigned to each gate
    x = model.addVars(n, m + 1, vtype=GRB.BINARY, name="x") # Number of real gates is num_gates; the last index is assumed to be the dummy gate

    # Constraints
    # Assign each activity to exactly one of its allowable gates
    for i in range(n):
        model.addConstr(quicksum(x[i, j] for j in range(m+1) if j in M[i]) == 1, name=f"Assign_{i}") 
        #'''can include dummy gate it in this quicksum (might leader to a tighter LP relaxation)'''

    # Non-overlapping constraint (1)
    for i in range(n):
        for j in range(n):
            if T[i][j] < 0:
                for k in range(m):
                    model.addConstr(x[i, k] + x[j, k] <= 1, name=f"Overlap_{i}_{j}_{k}")

    # Shadow restrictions (2)
    for (i, k, j, l) in shadow_restrictions:
        model.addConstr(x[i, k] + x[j, l] <= 1, name=f"Shadow_{i}_{j}")

    # Objective components (3)
    
    # Append significantly negative scores for the dummy gate
    P_star = [prefs + [-1000] for prefs in P]  # Add a very undesirable score for the dummy gate

    # z1: Minimize the negative sum of adjusted preferences
    z1 = - quicksum(P_star[i][j] * x[i, j] for i in range(n) for j in range(m + 1)) 
    #''' - quicksum(P_star[i][j] * x[i, j] for (i,j) in x). negative is important!'''
    
   
    # Define a new decision variable for tows between activities and their successors
    '''Good practice: first define ALL variables, then define ALL constraints, and ultimately define the objective function
    '''
    tows = model.addVars(n, m+1, vtype=GRB.BINARY, name="tows")
    #'''Towing to the dummy gate also needs to be possible (-> tighter lower bounds)'''

    # Calculate tows:
    for i in range(n):
        '''Each activity must have a successor, i.e. U[i] \in N for all i
        In the trivial case that would be the activity itself (->U[i] = i)'''
        for k in range(m+1):
            if U[i] != i:  # Check for a valid successor

                # If activity i is assigned to gate k and its successor U[i] to a different gate
                model.addConstr(tows[i, k] >= x[i, k] - x[U[i], k], name=f"TowIfDifferent_{i}_{k}")

                # model.addConstr(tows[i, k] >= x[i, k] - x[U[i], k], name=f"TowIfDifferent1_{i}_{k}")
                # model.addConstr(tows[i, k] >= x[U[i], k] - x[i, k], name=f"TowIfDifferent2_{i}_{k}")
                '''If I understand it correctly, it should be that tows[i,k] = 1 <=> flight associated with activity i
                needs to be towed away from gate k after activity i is done.
                First constraint: "if activity i is assigned to gate k, but its successor is not, we need to tow the plane 
                away from k after finishing i"
                second constraint: "if activity i is NOT assigned to gate k, but its success is, we need to tow the plane
                towards k after finishing i"
                The latter constraint set would then lead to every tow being counted twice if I'm not mistaken
                '''

    # Redefine z2 using the new tow variables
    z2 = quicksum(tows[i, k] for i in range(n) for k in range(m))

    # z3: Buffer time deficit
    # z3 = quicksum(max(t_max - T[i][j], 0) * x[i, k] * x[j, k] for i in range(n) for j in range(i +1, n) for k in range(m))
    '''This is a quadratic constraint. It can be linearized by introducing variables buffer[i,j] for activities i and j,
    bounding them from above by x[i,k]*(max(t_max - T_[i][j],0)) and x[j,k]*(max(t_max - T_[i][j],0)).
    '''

    buffer = model.addVars(n, n, vtype=GRB.BINARY, name="buffer")
    # TODO: Implement new buffer time constraint here


    
    
    # Combined objective
    model.setObjective(alpha1 * z1 + alpha2 * z2 + alpha3 * z3, GRB.MINIMIZE)
    model.optimize()

    return model, x, tows

In [ ]:
# Build the model
model, x = build_FGS_model(num_flights, num_gates, P, U, T, M, shadow_restrictions, alpha1, alpha2, alpha3, t_max)  # Receive model and decision variables

# Print the results
if model.status == GRB.OPTIMAL:
    assignments = model.getAttr('X', x)
    for i in range(num_flights):
        for j in range(num_gates):
            if assignments[i, j] > 0.5:
                print(f"Activity {i} assigned to Gate {j}")


### Draw a graph

Doesn't help

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def draw_cpp_graph(flights, gates, connections):
    """
    Draws a CPP graph with given flight and gate nodes and their connections in a hexagonal-like layout.
    
    Parameters:
    - flights (dict): Dictionary where keys are flight labels and values are dicts of gate connections with weights.
    - gates (list): List of gate labels.
    - connections (dict): Optional additional edges between flights with weights.
    """
    # Create a graph
    G = nx.Graph()

    # Add nodes for flights and gates
    G.add_nodes_from(flights.keys(), style='filled', fillcolor='red', shape='circle')
    G.add_nodes_from(gates, style='filled', fillcolor='blue', shape='square')

    # Add edges between flights and gates with preferences
    for flight, prefs in flights.items():
        for gate, weight in prefs.items():
            G.add_edge(flight, gate, weight=weight)

    # Add optional additional connections between flights
    for (f1, f2), weight in connections.items():
        G.add_edge(f1, f2, weight=weight)

    # Manually define positions for a hexagonal-like layout
    pos = {
        'F1': (-1, 1), 'F2': (0, 2), 'F3': (1, 1),  # Top vertex, and middle row vertices
        'F4': (0, 0),  # Bottom vertex of the upper hexagon (if we imagine a hexagon)
        'G1': (-0.5, -1), 'G2': (0.5, -1)  # Bottom vertices (gates)
    }

    # Draw the network
    nx.draw(G, pos, with_labels=True, node_color='skyblue', edge_color='gray', node_size=2000, font_size=15, font_weight='bold')

    # Draw edge labels
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

    plt.title("Clique Partitioning Problem Graph - Hexagonal Layout")
    plt.axis('off')  # Hide the axes
    plt.show()

# Example usage
flights = {
    'F1': {'G1': 50, 'G2': 30},
    'F2': {'G1': 40, 'G2': 60},
    'F3': {'G1': 30, 'G2': 70},
    'F4': {'G1': 20, 'G2': 80}
}
gates = ['G1', 'G2']
additional_connections = {('F1', 'F2'): 10, ('F2', 'F3'): 15}

draw_cpp_graph(flights, gates, additional_connections)
